In [17]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip3 install torchrl
!pip3 install "gymnasium[mujoco]"
!pip3 install -U 'mujoco-py<2.2,>=2.1'
!pip3 install tqdm

In [61]:
import torch
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
from torch import nn
from torchrl.envs import ObservationNorm, TransformedEnv, Compose, DoubleToFloat, StepCounter
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.utils import check_env_specs
from torchrl.modules import ProbabilisticActor, TanhNormal, ValueOperator

import matplotlib.pyplot as plt
from tqdm import tqdm

In [10]:
device = torch.device(
    'cuda:0' if torch.cuda.is_available() else
    'cpu'
)

In [11]:
MODEL_SIZE = 256
BATCH_SIZE = 1000
SUB_BATCH_SIZE = 64
LEARNING_RATE = 3e-4
MAX_GRAD_NORM = 1.0
MAX_FRAME_NUMBER = 50_000
EPOCHS = 10
CLIP_EPSILON = (0.2)
GAMMA = 0.99
LAMBDA = 0.95
ENTROPY_EPS = 1e-4

# Environment definition

In [42]:
observation_keys = ['observation']
base_env = GymEnv('InvertedDoublePendulum-v4', device=device)
env = TransformedEnv(
    base_env,
    Compose(
        # normalise onservations
        ObservationNorm(in_keys=observation_keys),
        DoubleToFloat(),
        StepCounter(),
    ),
)
env.transform[0].init_stats(num_iter=1000, reduce_dim=0, cat_dim=0)
print('env:', env)
print('normalisation shape:', env.transform[0].loc.shape)

env: TransformedEnv(
    env=GymEnv(env=InvertedDoublePendulum-v4, batch_size=torch.Size([]), device=cpu),
    transform=Compose(
            ObservationNorm(keys=['observation']),
            DoubleToFloat(in_keys=['observation'], out_keys=['observation'], in_keys_inv=[], out_keys_inv=[]),
            StepCounter(keys=[])))
normalisation shape: torch.Size([11])


In [43]:
# environment specs
print('observation spec:', env.observation_spec)
print('reward spec:', env.reward_spec)
print('inpput spec:', env.input_spec)
print('action spec:', env.action_spec)

observation spec: Composite(
    observation: UnboundedContinuous(
        shape=torch.Size([11]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([11]), device=cpu, dtype=torch.float32, contiguous=True),
            high=Tensor(shape=torch.Size([11]), device=cpu, dtype=torch.float32, contiguous=True)),
        device=cpu,
        dtype=torch.float32,
        domain=continuous),
    step_count: BoundedDiscrete(
        shape=torch.Size([1]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, contiguous=True),
            high=Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, contiguous=True)),
        device=cpu,
        dtype=torch.int64,
        domain=discrete),
    device=cpu,
    shape=torch.Size([]),
    data_cls=None)
reward spec: UnboundedContinuous(
    shape=torch.Size([1]),
    space=ContinuousBox(
        low=Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, contiguous=True),

In [44]:
# ensure specs are properly defined
check_env_specs(env)

2026-02-09 15:35:00,380 [torchrl][INFO]    check_env_specs succeeded! [END]


In [45]:
rollout = env.rollout(3)
rollout

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([3, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([3, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([3, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([3, 11]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([3, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                step_count: Tensor(shape=torch.Size([3, 1]), device=cpu, dtype=torch.int64, is_shared=False),
                terminated: Tensor(shape=torch.Size([3, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([3, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([3]),
            device=cpu,
            is_s

#Policy network definition

In [64]:
n_actions = env.action_spec.shape[-1]

actor_net = nn.Sequential(
    nn.LazyLinear(MODEL_SIZE, device=device),
    nn.Tanh(),
    nn.LazyLinear(MODEL_SIZE, device=device),
    nn.Tanh(),
    nn.LazyLinear(MODEL_SIZE, device=device),
    nn.Tanh(),
    nn.LazyLinear(2 * n_actions, device=device),
    NormalParamExtractor(),
)

norm_dict_keys = ['loc', 'scale']
policy = TensorDictModule(actor_net, in_keys=observation_keys, out_keys=norm_dict_keys)
policy_net = ProbabilisticActor(
    module=policy,
    spec=env.action_spec,
    in_keys=norm_dict_keys,
    distribution_class=TanhNormal,
    distribution_kwargs={
        'low': env.action_spec.space.low,
        'high': env.action_spec.space.high,
    },
    return_log_prob=True,
)
policy_net

ProbabilisticActor(
    module=ModuleList(
      (0): TensorDictModule(
          module=Sequential(
            (0): LazyLinear(in_features=0, out_features=256, bias=True)
            (1): Tanh()
            (2): LazyLinear(in_features=0, out_features=256, bias=True)
            (3): Tanh()
            (4): LazyLinear(in_features=0, out_features=256, bias=True)
            (5): Tanh()
            (6): LazyLinear(in_features=0, out_features=2, bias=True)
            (7): NormalParamExtractor(
              (scale_mapping): biased_softplus()
            )
          ),
          device=cpu,
          in_keys=['observation'],
          out_keys=['loc', 'scale'])
      (1): SafeProbabilisticModule(
          in_keys=['loc', 'scale'],
          out_keys=['action', 'action_log_prob'],
          distribution_class=<class 'torchrl.modules.distributions.continuous.TanhNormal'>, 
          distribution_kwargs={'low': tensor([-1.]), 'high': tensor([1.])}),
          default_interaction_type=deter

In [65]:
print("Running policy:", policy_net(env.reset()))

Running policy: TensorDict(
    fields={
        action: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        action_log_prob: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        loc: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        observation: Tensor(shape=torch.Size([11]), device=cpu, dtype=torch.float32, is_shared=False),
        scale: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        step_count: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)


#Value network definition

In [63]:
value = nn.Sequential(
    nn.LazyLinear(MODEL_SIZE, device=device),
    nn.Tanh(),
    nn.LazyLinear(MODEL_SIZE, device=device),
    nn.Tanh(),
    nn.LazyLinear(MODEL_SIZE, device=device),
    nn.Tanh(),
    nn.LazyLinear(n_actions, device=device),
)
value_net = ValueOperator(
    module=value,
    in_keys=observation_keys,
)
value_net

ValueOperator(
    module=Sequential(
      (0): LazyLinear(in_features=0, out_features=256, bias=True)
      (1): Tanh()
      (2): LazyLinear(in_features=0, out_features=256, bias=True)
      (3): Tanh()
      (4): LazyLinear(in_features=0, out_features=256, bias=True)
      (5): Tanh()
      (6): LazyLinear(in_features=0, out_features=1, bias=True)
    ),
    device=cpu,
    in_keys=['observation'],
    out_keys=['state_value'])

In [66]:
print("Running value:", value_net(env.reset()))

Running value: TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        observation: Tensor(shape=torch.Size([11]), device=cpu, dtype=torch.float32, is_shared=False),
        state_value: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        step_count: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)
